In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
from src.models import Ensemble, RandomForest
from src.test import load_processed_data, load_pretrained_model, load_best_params

DATASET = 'NYMEX-NG'

In [ ]:
class MixedTypeEnsemble(Ensemble):
    def __init__(self, model_type, train_lags, train_target):
        for model_type in ['NN', 'CNN', 'LSTM', 'RandomForest']:
            if model_type in ['NN', 'CNN', 'LSTM']:
                model = load_pretrained_model(model_type, DATASET)
            else:
                # Load best para mmax_features:
                _, param = load_best_params(model_type, DATASET)
                model = RandomForest(n_estimators=10000, random_state=0, max_features=param)
                model.train(train_lags, train_target)

: 

In [ ]:
train_lags, train_target, train_dates = load_processed_data(DATASET, partition='train')
val_lags, val_target, val_dates = load_processed_data(DATASET, partition='val')
test_lags, test_target, test_dates = load_processed_data(DATASET, partition='test')



# Define number of periods per year to compute performance metrics:
if DATASET in ['BTC-USD', 'ETH-USD', 'XRP-USD']:
    n_periods = 365
else:
    n_periods = 252

# Plot cumulative sum of model's log-returns:
fig = plt.figure(figsize=(9, 6))
fig.suptitle('Performances of {} on {}'.format(model_type, dataset), fontsize=14)
plt.subplots_adjust(wspace= 0.25, hspace= 0.25)

sub1 = fig.add_subplot(2,2,(1,2))
model.test(train_lags, train_target, n_periods)
sub1.plot(train_dates, np.cumsum(model.log_returns))
sub1.plot(train_dates, np.cumsum(train_target.to('cpu')))
sub1.grid(True)
sub1.set_ylabel('Cumulated log-performances')
sub1.set_title('Train set')
sub1.legend([model_type, 'Baseline'], loc="upper left")

sub2 = fig.add_subplot(2,2,3)
model.test(val_lags, val_target, n_periods)
sub2.plot(val_dates, np.cumsum(model.log_returns))
sub2.plot(val_dates, np.cumsum(val_target).to('cpu'))
sub2.tick_params(labelrotation=45)
sub2.grid(True)
sub2.set_ylabel('Cumulated log-performances')
sub2.set_title('Validation set')

sub3 = fig.add_subplot(2,2,4)
model.test(test_lags, test_target, n_periods)
sub3.plot(test_dates, np.cumsum(model.log_returns))
sub3.plot(test_dates, np.cumsum(test_target.to('cpu')))
sub3.tick_params(labelrotation=45)
sub3.grid(True)
sub3.set_title('Test set')

plt.show()